<a href="https://colab.research.google.com/github/weieric1001/nfsepsisanalysis/blob/main/tools/modelTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import pickle


In [2]:
# 呼叫雲端硬碟
import pandas as pd
from google.colab import drive
drive.mount("/content/drive")

# 讀取資料集
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/aa5.csv')
print(df)


Mounted at /content/drive
      sea    wbc     crp   seg  band  nf
0       1  15000   26.38  91.5   8.0   1
1       1  24100  174.27  83.0   2.0   1
2       1   3500    1.94  60.0   2.0   1
3       1  16500  191.54  88.8   0.0   1
4       1  18600  137.31  79.5   7.0   1
...   ...    ...     ...   ...   ...  ..
1410    1   9500  110.58  83.0   0.0   0
1411    0   7000    2.85  71.8   0.0   0
1412    0   5900   21.04  75.0   0.0   0
1413    1  14700   19.87  78.7   0.0   0
1414    0   6700   15.98  66.2   0.0   0

[1415 rows x 6 columns]


In [3]:
# 讀取資料集
df = pd.read_csv('document/aa5.csv')
print(df)


FileNotFoundError: ignored

In [4]:
# 顯示資料集的描述資料
print('資料集的描述：')
print(df.describe())


資料集的描述：
               sea            wbc          crp          seg         band  \
count  1415.000000    1415.000000  1415.000000  1415.000000  1415.000000   
mean      0.178092   11340.508834    70.930066    75.215760     0.726007   
std       0.382725    6256.728228    84.992794    11.570522     2.912236   
min       0.000000    1100.000000     0.500000     5.300000     0.000000   
25%       0.000000    7500.000000     8.115000    68.000000     0.000000   
50%       0.000000   10100.000000    35.250000    77.000000     0.000000   
75%       0.000000   14000.000000   110.080000    84.000000     0.000000   
max       1.000000  108000.000000   709.310000    96.600000    31.500000   

                nf  
count  1415.000000  
mean      0.122968  
std       0.328517  
min       0.000000  
25%       0.000000  
50%       0.000000  
75%       0.000000  
max       1.000000  


In [11]:
X_train = df.iloc[:, :5]
Y_train = df.iloc[:, 5]
X_train_class_0 = X_train[Y_train == 0]
X_train_class_1 = X_train[Y_train == 1]
X_train_class_0_under = X_train_class_0.sample(X_train_class_1.shape[0])
X_train_mean = X_train_class_0_under.mean(axis=0)
X_train_std = X_train_class_0_under.std(axis=0)
print(X_train_mean)
print(X_train_std)

sea         0.137931
wbc     10877.586207
crp        51.087874
seg        74.245977
band        0.266092
dtype: float64
sea        0.345823
wbc     4898.568026
crp       63.134101
seg       11.010130
band       1.293983
dtype: float64


In [12]:
# 顯示資料集的資訊
print('資料集的資訊：')
print(df.info())


資料集的資訊：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1415 entries, 0 to 1414
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   sea     1415 non-null   int64  
 1   wbc     1415 non-null   int64  
 2   crp     1415 non-null   float64
 3   seg     1415 non-null   float64
 4   band    1415 non-null   float64
 5   nf      1415 non-null   int64  
dtypes: float64(3), int64(3)
memory usage: 66.5 KB
None


In [14]:
# 顯示沒有資料的筆數
print('沒有資料的筆數：')
print(df.isnull().sum())


沒有資料的筆數：
sea     0
wbc     0
crp     0
seg     0
band    0
nf      0
dtype: int64


In [ ]:
# 資料處理1 Decision tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

clf = DecisionTreeClassifier(
    criterion='entropy', max_depth=5, min_samples_leaf=9)
scaler = StandardScaler()

# 編碼後的X與Y
X_train = df.iloc[:, :5]
Y_train = df.iloc[:, 5]

# Class count
count_class_0, count_class_1 = Y_train.value_counts()

# Divide by class
X_train_class_0 = X_train[Y_train == 0]
X_train_class_1 = X_train[Y_train == 1]
X_train_class_0_under = X_train_class_0.sample(
    X_train_class_1.shape[0])  # 174
X_train_under = pd.concat([X_train_class_0_under, X_train_class_1], axis=0)
X_train_final = X_train_under
Y_train_final = Y_train[X_train_under.index]

# 資料標準化
print("Data Rescaling ...")
X_train_final = pd.DataFrame(scaler.fit_transform(X_train_final))
print('X_train_final.shape：{}'.format(X_train_final.shape))
print('Y_train_final.shape：{}'.format(Y_train_final.shape))

# 訓練模型1 Decision tree
print("Training ...")
clf.fit(X_train_final, Y_train_final)

# 評估模型1 Decision tree
print("Testing ...")
# 計算訓練資料集的準確度
accuracy = clf.score(X_train_final, Y_train_final)
print("Decision tree 訓練資料集的準確度 = {:.4f}".format(accuracy))

# 儲存模型
model_file_name = 'model/decisionTree.pickle'
with open(model_file_name, 'wb') as f:
    pickle.dump(clf, f)


Data Rescaling ...
X_train_final.shape：(348, 5)
Y_train_final.shape：(348,)
Training ...
Testing ...
Decision tree 訓練資料集的準確度 = 0.8477


In [ ]:
# 資料處理2 Random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

rf = RandomForestClassifier(
    n_estimators=750, max_depth=20, criterion='entropy', max_features='sqrt')
scaler = StandardScaler()

# 編碼後的X與Y
X_train = df.iloc[:, :5]
Y_train = df.iloc[:, 5]

# Class count
count_class_0, count_class_1 = Y_train.value_counts()

# Divide by class
X_train_class_0 = X_train[Y_train == 0]
X_train_class_1 = X_train[Y_train == 1]
X_train_class_0_under = X_train_class_0.sample(
    X_train_class_1.shape[0])  # 174
X_train_under = pd.concat([X_train_class_0_under, X_train_class_1], axis=0)
X_train_final = X_train_under
Y_train_final = Y_train[X_train_under.index]

# 資料標準化
print("Data Rescaling ...")
X_train_final = pd.DataFrame(scaler.fit_transform(X_train_final))
print('X_train_final.shape：{}'.format(X_train_final.shape))
print('Y_train_final.shape：{}'.format(Y_train_final.shape))

# 訓練模型2 Random forest
print("Training ...")
rf.fit(X_train_final, Y_train_final)

# 評估模型2 Random forest
print("Testing ...")
# 計算訓練資料集的準確度
accuracy = rf.score(X_train_final, Y_train_final)
print("Random forest 訓練資料集的準確度 = {:.4f}".format(accuracy))

# 儲存模型
model_file_name = 'model/randomForest.pickle'
with open(model_file_name, 'wb') as f:
    pickle.dump(rf, f)


Data Rescaling ...
X_train_final.shape：(348, 5)
Y_train_final.shape：(348,)
Training ...
Testing ...
Random forest 訓練資料集的準確度 = 1.0000


In [41]:
# 資料處理3 Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

logreg = LogisticRegression(penalty='l2', C=0.01)
scaler = StandardScaler()

# 編碼後的X與Y
X_train = df.iloc[:, :5]
Y_train = df.iloc[:, 5]

# Class count
count_class_0, count_class_1 = Y_train.value_counts()

# Divide by class
X_train_class_0 = X_train[Y_train == 0]
X_train_class_1 = X_train[Y_train == 1]
X_train_class_0_under = X_train_class_0.sample(
    X_train_class_1.shape[0])  # 174
X_train_under = pd.concat([X_train_class_0_under, X_train_class_1], axis=0)
X_train_final = X_train_under
Y_train_final = Y_train[X_train_under.index]

# 資料標準化
print("Data Rescaling ...")
X_train_mean = X_train_final.mean(axis=0)
X_train_std = X_train_final.std(axis=0)
print(X_train_mean)
print(X_train_std)
X_train_final = pd.DataFrame(scaler.fit_transform(X_train_final))
print('X_train_final.shape：{}'.format(X_train_final.shape))
print('Y_train_final.shape：{}'.format(Y_train_final.shape))

# 訓練模型3 Logistic Regression
print("Training ...")
logreg.fit(X_train_final, Y_train_final)

# 評估模型3 Logistic Regression
print("Testing ...")
# 計算訓練資料集的準確度
X_train = pd.DataFrame(scaler.fit_transform(X_train))
accuracy = logreg.score(X_train_final, Y_train_final)
print("Logistic Regression 訓練資料集的準確度 = {:.4f}".format(accuracy))

# 儲存模型
model_file_name = 'logisticregression.pickle'
with open(model_file_name, 'wb') as f:
    pickle.dump(logreg, f)

Data Rescaling ...
sea         0.255747
wbc     12546.839080
crp       101.779655
seg        77.377299
band        2.160057
dtype: float64
sea        0.436909
wbc     6951.471663
crp      102.112756
seg       11.340239
band       5.126581
dtype: float64
X_train_final.shape：(348, 5)
Y_train_final.shape：(348,)
Training ...
Testing ...
Logistic Regression 訓練資料集的準確度 = 0.8333


In [47]:
# 資料處理5 Support Vector Machine
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

model = SVC(kernel='rbf', C=10, gamma=0.0001, cache_size=1000, probability=True)
scaler = StandardScaler()

# 編碼後的X與Y
X_train = df.iloc[:, :5]
Y_train = df.iloc[:, 5]

# Class count
count_class_0, count_class_1 = Y_train.value_counts()

# Divide by class
print("Data Rescaling ...")
X_train_class_0 = X_train[Y_train == 0]
X_train_class_1 = X_train[Y_train == 1]
X_train_class_0_under = X_train_class_0.sample(
    X_train_class_1.shape[0])  # 174
X_train_under = pd.concat([X_train_class_0_under, X_train_class_1], axis=0)
X_train_final = X_train_under
Y_train_final = Y_train[X_train_under.index]

# 資料標準化
X_train_mean = X_train_final.mean(axis=0)
X_train_std = X_train_final.std(axis=0)
print(X_train_mean)
print(X_train_std)
X_train_final = pd.DataFrame(scaler.fit_transform(X_train_final))
print('X_train_final.shape：{}'.format(X_train_final.shape))
print('Y_train_final.shape：{}'.format(Y_train_final.shape))

# 訓練模型5 Support Vector Machine
print("Training ...")
model.fit(X_train_final, Y_train_final)

# 評估模型5 Support Vector Machine
print("Testing ...")
# 計算訓練資料集的準確度
X_train = pd.DataFrame(scaler.fit_transform(X_train))
accuracy = model.score(X_train_final, Y_train_final)
print("Support Vector Machine 訓練資料集的準確度 = {:.4f}".format(accuracy))

# 儲存模型
model_file_name = 'supportVectorMachine.pickle'
with open(model_file_name, 'wb') as f:
    pickle.dump(model, f)

Data Rescaling ...
sea         0.238506
wbc     12409.252874
crp        96.047414
seg        77.022701
band        2.236782
dtype: float64
sea        0.426784
wbc     6895.573196
crp      101.699342
seg       10.816080
band       5.159527
dtype: float64
X_train_final.shape：(348, 5)
Y_train_final.shape：(348,)
Training ...
Testing ...
Support Vector Machine 訓練資料集的準確度 = 0.8190


In [ ]:
# 讀取模型
# model_file_name = 'model/decisionTree.pickle'
# model_file_name = 'model/randomForest.pickle'
# model_file_name = 'model/logisticregression.pickle'
# model_file_name = 'model/supportVectorMachine.pickle'
with open(model_file_name, 'rb') as f:
    model = pickle.load(f)

X = df.iloc[:, :5]
Y = df.iloc[:, 5]

# 計算全部資料集的準確度
X = pd.DataFrame(scaler.fit_transform(X))
accuracy = model.score(X, Y)
print("全部資料集的準確度 = {:.4f}".format(accuracy))

全部資料集的準確度 = 0.7074
